In [1]:
!pip install -q "maestro[qwen_2_5_vl]==1.1.0rc2"
!pip install -q "supervision==0.26.0rc4"    

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.1.0+cu118 requires torch==2.1.0, but you have torch 2.7.0 which is incompatible.

[notice] A new release of pip is available: 23.3.1 -> 25.1
[notice] To update, run: python -m pip install --upgrade pip
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
maestro 1.1.0rc2 requires supervision<0.26.0,>=0.20.0, but you have supervision 0.26.0rc4 which is incompatible.

[notice] A new release of pip is available: 23.3.1 -> 25.1
[notice] To update, run: python -m pip install --upgrade pip


In [2]:
import os
from PIL import Image
from typing import List, Tuple, Optional
import concurrent.futures
import supervision as sv
import numpy as np


In [3]:
# Constants
MODEL_ID_OR_PATH = "Qwen/Qwen2.5-VL-7B-Instruct"
MIN_PIXELS = 512 * 28 * 28
MAX_PIXELS = 2048 * 28 * 28
# SYSTEM_MESSAGE = None
# PROMPT = """"""
SYSTEM_MESSAGE = None

PROMPT =  "Outline the position of each person and output all the coordinates in JSON format."

NUM_WORKERS = 2 # Adjust based on your GPU memory and performance needs

In [4]:
# Load model (should be done once)
from maestro.trainer.models.qwen_2_5_vl.checkpoints import load_model, OptimizationStrategy
from maestro.trainer.models.qwen_2_5_vl.inference import predict_with_inputs
from maestro.trainer.models.qwen_2_5_vl.loaders import format_conversation
from maestro.trainer.common.utils.device import parse_device_spec
from qwen_vl_utils import process_vision_info

In [5]:
processor, model = load_model(
    model_id_or_path=MODEL_ID_OR_PATH,
    optimization_strategy=OptimizationStrategy.NONE,
    min_pixels=MIN_PIXELS,
    max_pixels=MAX_PIXELS
)

preprocessor_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


tokenizer_config.json:   0%|          | 0.00/5.70k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

...

model-00003-of-00005.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

model-00004-of-00005.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

model-00001-of-00005.safetensors:   0%|          | 0.00/3.90G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/216 [00:00<?, ?B/s]

In [6]:
def run_qwen_2_5_vl_inference(
    model,
    processor,
    image: Image.Image,
    prompt: str,
    system_message: Optional[str] = None,
    device: str = "auto",
    max_new_tokens: int = 1024,
) -> Tuple[str, Tuple[int, int]]:
    device = parse_device_spec(device)
    conversation = format_conversation(image=image, prefix=prompt, system_message=system_message)
    text = processor.apply_chat_template(conversation, tokenize=False, add_generation_prompt=True)
    image_inputs, _ = process_vision_info(conversation)

    inputs = processor(
        text=text,
        images=image_inputs,
        return_tensors="pt",
    )

    input_h = inputs['image_grid_thw'][0][1] * 14
    input_w = inputs['image_grid_thw'][0][2] * 14

    response = predict_with_inputs(
        **inputs,
        model=model,
        processor=processor,
        device=device,
        max_new_tokens=max_new_tokens
    )[0]

    return response, (input_w, input_h)
     

In [7]:


# Configuration
IMAGE_PATHS = ["image_1.jpg", "image_2.jpg", "image_3.jpg", "image_4.jpg" , "image_5.jpg","image_6.jpg","image_7.jpg","image_8.jpg",]  
OUTPUT_DIR = "annotated_results"
SYSTEM_MESSAGE = None
PROMPT = """Outline the position of all the person that are working on some height and output all the coordinates in JSON format."""

# Create output directory if it doesn't exist
os.makedirs(OUTPUT_DIR, exist_ok=True)

def process_and_save_images(image_paths: List[str]):
    """Process multiple images and save annotated results"""
    box_annotator = sv.BoxAnnotator(color_lookup=sv.ColorLookup.INDEX)
    label_annotator = sv.LabelAnnotator(color_lookup=sv.ColorLookup.INDEX)

    for image_path in image_paths:
        try:
            # Load image
            image = Image.open(image_path)
            resolution_wh = image.size
            
            # Run inference
            response, input_wh = run_qwen_2_5_vl_inference(
                model=model,
                processor=processor,
                image=image,
                prompt=PROMPT,
                system_message=SYSTEM_MESSAGE
            )
            
            print(f"Results for {image_path}:")
            print(response)

            # Parse detections
            detections = sv.Detections.from_vlm(
                vlm=sv.VLM.QWEN_2_5_VL,
                result=response,
                input_wh=input_wh,
                resolution_wh=resolution_wh
            )

            # Annotate image
            annotated_image = np.array(image.copy())
            annotated_image = box_annotator.annotate(scene=annotated_image, detections=detections)
            annotated_image = label_annotator.annotate(scene=annotated_image, detections=detections)
            
            # Convert back to PIL Image and save
            output_path = os.path.join(OUTPUT_DIR, f"annotated_{os.path.basename(image_path)}")
            Image.fromarray(annotated_image).convert('RGB').save(output_path)
            print(f"Saved annotated image to {output_path}")

        except Exception as e:
            print(f"Error processing {image_path}: {str(e)}")

if __name__ == "__main__":
    process_and_save_images(IMAGE_PATHS)
    print("Batch processing completed!")

Error processing image_1.jpg: [Errno 2] No such file or directory: 'image_1.jpg'
Error processing image_2.jpg: [Errno 2] No such file or directory: 'image_2.jpg'
Error processing image_3.jpg: [Errno 2] No such file or directory: 'image_3.jpg'
Error processing image_4.jpg: [Errno 2] No such file or directory: 'image_4.jpg'
Error processing image_5.jpg: [Errno 2] No such file or directory: 'image_5.jpg'
Error processing image_6.jpg: [Errno 2] No such file or directory: 'image_6.jpg'
Error processing image_7.jpg: [Errno 2] No such file or directory: 'image_7.jpg'
Error processing image_8.jpg: [Errno 2] No such file or directory: 'image_8.jpg'
Batch processing completed!


USING THREADING OPERATION !!!!

In [8]:
from concurrent.futures import ThreadPoolExecutor, as_completed

In [29]:
def process_single_image(image_path: str):
    """Process and annotate a single image."""
    box_annotator = sv.BoxAnnotator(color_lookup=sv.ColorLookup.INDEX)
    label_annotator = sv.LabelAnnotator(color_lookup=sv.ColorLookup.INDEX)

    try:
        PROMPT = """Outline the position of all the person that are working on some height and output all the coordinates in STRICTLY in JSON format."""

        # Load image
        image = Image.open(image_path)
        resolution_wh = image.size

        # Run inference
        response, input_wh = run_qwen_2_5_vl_inference(
            model=model,
            processor=processor,
            image=image,
            prompt=PROMPT,
            system_message=SYSTEM_MESSAGE
        )

        print(f"Results for {image_path}:")
        print(response)

        # Parse detections
        detections = sv.Detections.from_vlm(
            vlm=sv.VLM.QWEN_2_5_VL,
            result=response,
            input_wh=input_wh,
            resolution_wh=resolution_wh
        )

        # Annotate image
        annotated_image = np.array(image.copy())
        annotated_image = box_annotator.annotate(scene=annotated_image, detections=detections)
        annotated_image = label_annotator.annotate(scene=annotated_image, detections=detections)

        # Save result
        output_path = os.path.join(OUTPUT_DIR, f"annotated_{os.path.basename(image_path)}")
        Image.fromarray(annotated_image).convert('RGB').save(output_path)
        print(f"Saved annotated image to {output_path}")

    except Exception as e:
        print(f"Error processing {image_path}: {str(e)}")


In [34]:
def process_and_save_images(image_paths: List[str], max_workers: int = 1):
    """Parallel image processing using threads."""
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = [executor.submit(process_single_image, path) for path in image_paths]
        for future in as_completed(futures):
            future.result()  # to raise any exceptions if occurred

In [35]:
IMAGE_PATHS = ["image_1.jpg", "image_2.jpg", "image_3.jpg", "image_4.jpg" , "image_5.jpg","image_6.jpg","image_7.jpg","image_8.jpg",]  

In [38]:
if __name__ == "__main__":
    process_and_save_images(IMAGE_PATHS, max_workers=1)  # 2 threads: each handles half
    print("Batch processing completed!")


Results for image_1.jpg:
```json
[
	{"bbox_2d": [153, 109, 286, 334], "label": "person that are working on some height"},
	{"bbox_2d": [197, 73, 357, 242], "label": "person that are working on some height"},
	{"bbox_2d": [481, 95, 551, 245], "label": "person that are working on some height"},
	{"bbox_2d": [626, 116, 677, 269], "label": "person that are working on some height"}
]
```
Saved annotated image to annotated_results/annotated_image_1.jpg
Results for image_2.jpg:
```json
[
	{"bbox_2d": [395, 0, 471, 151], "label": "person that are working on some height"},
	{"bbox_2d": [480, 108, 530, 246], "label": "person that are working on some height"},
	{"bbox_2d": [414, 263, 468, 408], "label": "person that are working on some height"},
	{"bbox_2d": [462, 390, 524, 550], "label": "person that are working on some height"},
	{"bbox_2d": [530, 452, 582, 550], "label": "person that are working on some height"}
]
```
Saved annotated image to annotated_results/annotated_image_2.jpg
Results for

In [9]:
def process_single_image(image_path: str):
    """Process and annotate a single image."""
    box_annotator = sv.BoxAnnotator(color_lookup=sv.ColorLookup.INDEX)
    label_annotator = sv.LabelAnnotator(color_lookup=sv.ColorLookup.INDEX)

    try:
        PROMPT = """Detect and outline the position of all persons working at height. Additionally, identify if any worker is (1) smoking a cigarette or (2) using a mobile phone. Output all relevant coordinates and detected actions STRICTLY in JSON format."""

        # Load image
        image = Image.open(image_path)
        resolution_wh = image.size

        # Run inference
        response, input_wh = run_qwen_2_5_vl_inference(
            model=model,
            processor=processor,
            image=image,
            prompt=PROMPT,
            system_message=SYSTEM_MESSAGE
        )

        print(f"Results for {image_path}:")
        print(response)

        # Parse detections
        detections = sv.Detections.from_vlm(
            vlm=sv.VLM.QWEN_2_5_VL,
            result=response,
            input_wh=input_wh,
            resolution_wh=resolution_wh
        )

        # Annotate image
        annotated_image = np.array(image.copy())
        annotated_image = box_annotator.annotate(scene=annotated_image, detections=detections)
        annotated_image = label_annotator.annotate(scene=annotated_image, detections=detections)

        # Save result
        output_path = os.path.join(OUTPUT_DIR, f"annotated_{os.path.basename(image_path)}")
        Image.fromarray(annotated_image).convert('RGB').save(output_path)
        print(f"Saved annotated image to {output_path}")

    except Exception as e:
        print(f"Error processing {image_path}: {str(e)}")


In [10]:
def process_and_save_images(image_paths: List[str], max_workers: int = 1):
    """Parallel image processing using threads."""
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = [executor.submit(process_single_image, path) for path in image_paths]
        for future in as_completed(futures):
            future.result()  # to raise any exceptions if occurred

In [11]:
IMAGE_PATHS = [
    "imagee_1.jpg", "imagee_2.jpg", "imagee_3.jpg", "imagee_4.jpg", "imagee_5.jpg",
    "imagee_6.jpg", "imagee_7.jpg", "imagee_8.jpg", "imagee_9.jpg", "imagee_10.jpg",
    "imagee_11.jpg", "imagee_12.jpg", "imagee_13.jpg", "imagee_14.jpg", "imagee_15.jpg",
    "imagee_16.jpg", "imagee_17.jpg", "imagee_18.jpg", "imagee_19.jpg", "imagee_20.jpg"
]


In [12]:
if __name__ == "__main__":
    process_and_save_images(IMAGE_PATHS, max_workers=1)  # 2 threads: each handles half
    print("Batch processing completed!")


Results for imagee_1.jpg:
```json
[
	{"bbox_2d": [283, 397, 635, 1007], "label": "persons working at height"},
	{"bbox_2d": [370, 494, 486, 612], "label": "(2) using a mobile phone"}
]
```
Saved annotated image to annotated_results/annotated_imagee_1.jpg
Results for imagee_2.jpg:
```json
[
	{"bbox_2d": [500, 34, 716, 800], "label": "persons working at height"},
	{"bbox_2d": [872, 630, 907, 719], "label": "persons working at height"},
	{"bbox_2d": [919, 681, 945, 720], "label": "persons working at height"},
	{"bbox_2d": [500, 34, 716, 800], "label": "(2) using a mobile phone"}
]
```
Saved annotated image to annotated_results/annotated_imagee_2.jpg
Results for imagee_3.jpg:
```json
[
	{"bbox_2d": [30, 0, 394, 415], "label": "persons working at height"},
	{"bbox_2d": [486, 87, 757, 415], "label": "persons working at height"},
	{"bbox_2d": [198, 76, 230, 129], "label": "(2) using a mobile phone"}
]
```
Saved annotated image to annotated_results/annotated_imagee_3.jpg
Results for imagee_4.j